# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the FEP package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
*default-pathname-defaults*

In [ ]:
(ql:quickload :fep)

In [ ]:
(setup-default-paths)

In [ ]:
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

In [ ]:
(source "leaprc.ff14SB.redq")

In [ ]:
(source "leaprc.gaff")

In [ ]:
(list-force-fields)

# Start a new FEP calculation

In [ ]:
(:= *feps* (make-instance 'fep::fep-calculation))

# Import sketch - use Chemdraw

## Open chemdraw and add more functional groups if you want

In [ ]:
(png-from-file "ligands.png" )

In [ ]:
(:= *sk* (handler-bind ((warning #'muffle-warning))
             (with-open-file (fin (open "ligands.cdxml" :direction :input))
             (chem:make-chem-draw fin :add-hydrogens nil))))

In [ ]:
(fep:setup-ligands *feps* *sk*)

In [ ]:
(:= *v* (show (fep:layout-ligands *feps*))) *v*

In [ ]:
(:= *tests* (list (cons :c1 (lambda (a) (eq (chem:get-name a) :c1)))
                  (cons :c3 (lambda (a) (eq (chem:get-name a) :c3)))
                  (cons :c5 (lambda (a) (eq (chem:get-name a) :c5)))))
(:= *pick* (chem:compile-smarts 
             "[C:6]1~[C<c1>:1]~[C:2]~[C<c3>:3]~[C:4]~[C<c5>:5]1" :tests *tests*))

In [ ]:
(show (fep:molecule (find-if (lambda (x) (eq (fep:name x) :AA)) (fep:ligands *feps*))))

# Load the lysozyme PDB file

In [ ]:
(start-swank)

In [ ]:
(symbol-package (find-symbol "LOAD-PDB"))

In [ ]:
(apropos "load-pdb")

In [ ]:
(:= *lysozyme* (load-pdb "181L_mod.pdb"))

In [ ]:
(cando:build-unbuilt-hydrogens *lysozyme*)

In [ ]:
(simple-build-unbuilt-atoms *lysozyme*)

In [ ]:
(:= *v* (show *lysozyme*)) *v*

In [ ]:
(nglv::add-representation *v* "ball+stick" :selection "all")

In [ ]:
(simple-build-unbuilt-atoms *lysozyme*)

In [ ]:
*lysozyme*

In [ ]:
(pushnew *lysozyme* (fep:receptors *feps*))

# Load the ligands

In [ ]:
*default-pathname-defaults*

In [ ]:
(load-off "phen.lib")
(load-off "benz.lib")

In [ ]:
(:= *ligs* (load-pdb "bnz_phn.pdb"))

In [ ]:
(simple-build-unbuilt-atoms *ligs*)
(build-unbuilt-hydrogens *ligs*)

In [ ]:
(show *ligs*)

## Print the fixed atoms of the template ligand

In [ ]:
(alexandria:hash-table-alist (fep:pattern-atoms *pick* *ligs*))

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(fep:build-ligands *feps*)

In [ ]:
(mapcar #'fep:molecule (fep:ligands *feps*))

In [ ]:
(show (fep:layout-ligands *feps* :accessor 'fep::molecule))

# Pose the new ligands onto the template ligand

In [ ]:
(fep::pose-ligands-using-pattern *feps* *pick* *ligs*)

In [ ]:
(:= vreceptor (show *lysozyme*)) vreceptor

In [ ]:
(:= *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (fep::molecule (second (fep:ligands *feps*))))
(:= *struct* (make-instance 'cando-structure :matter *moveable-agg*))
(nglv::add-structure vreceptor *struct*)

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(fep::build-initial-jobs *feps* :connections 2 :stages 3 :windows 11)

In [ ]:
(fep:jobs *feps*)

In [ ]:
(graph::save-graph *feps*)

# Generate the calculations

In [ ]:
(start-swank)

In [ ]:
(:= *work-list* (fep::generate-jobs *feps*))

In [ ]:
*work-list*

In [ ]:
(with-open-file (sout "/tmp/graph.dot" :direction :output)
    (fepdot::draw-graph-stream (list *work-list*) sout))

In [ ]:
(ext:system "dot -Tpdf -O /tmp/graph.dot")

In [ ]:
(ext:system "open -n /tmp/graph.dot.pdf")